# Single-particle motion workbook

This notebook provides an investigation into single-particle motion within a laser.  It was developed by Kyle Miller based on notes from Professor W.B. Mori's graduate class in which exact analytical solutions of the particle motion are obtained. An improvement for the expansion in these notes was found by Kyle Miller and is presented here. Some of the calculations have also been presented by J. Yang, R. Craxton, and M. Haines, "Explicit general solutions to relativistic electron dynamics in plane-wave electromagnetic fields and simulations of ponderomotive acceleration," Plasma Phys. Contr. Fusion **53**, 125006 (2011).  This notebook can also be used to explore the accuracy of the equation of motion (Boris push and extensions) used in particle-in-cell (PIC) simulations for the motion of a single-particle in the analytical fields of a plane wave.   Parameters such as the laser intensity as well as initial phase can be varied to compare results against theory.

Consider a plane wave and linearly polarized laser with a profile

$\vec{A}=\hat{x} A(\xi) \cos (k_0 \xi+\phi), \qquad \vec{E} = \hat{x}\left[ -\frac{dA}{d\xi} \cos (k_0 \xi+\phi) + A(\xi) k_0 \sin (k_0 \xi+\phi) \right], \qquad \vec{k}=\hat{z}k_0$.

where $\xi\equiv z-ct$. In class we showed that for a single particle interacting with such a wave two quanities are conserved: $\gamma-p_z/mc$ and $p_x+qA/c$ (the canonical momemtum in the translationally invariant direction).  If we initialize a particle in a laser with constant amplitude $A(\xi)=A_0$ and with zero initial velocity at $x=z=0$ and $\phi=\pi/2$ (or any $\phi=(4n+1)\pi/2)$, then we showed that you can express the longitudinal motion $z(t)$ as a drift plus some oscillation.  An explicit solution for this was given in the class notes as

$z(t) = v_d t + \frac{1}{k_0} \sum_{n=1}^{\infty} \frac{(-1)^{n}}{n!} J_n\left(\frac{n v_d}{c}\right) \sin\left(2n\Omega_0t\right)$,

where $v_d = \frac{c A_0^2}{4+A_0^2}$ and $\Omega_0=k_0 c\left( 1-\frac{v_d}{c} \right)$.  One can also obtain explicit time solutions for $x$, $p_z$, and $p_x$, which are not given here. Following the procedures in the notes,   one can extend the theory to include cases where particles do not start at rest and/or start at different phases of the laser. For the rest of this workbook, analytical solutions will be taken from Yang et al., which allow for general initial conditions of the particle and phase of the laser.

Below you can run OSIRIS simulations in which a laser propagates towards a particle that is either starting at rest or already in motion. The trajectories of various quantities for the particles are plotted along with theory curves. You can choose your own parameters for the laser amplitude ($A_0$), initial phase deviation from $\pi/2$ (could be thought of as $\delta \phi$; in input deck below $\phi$ is given in *units of degrees*), and initial longitudinal proper velocity of the particle ($u_{z0}=\gamma v_{z0}$).  You will also be able to run with various field solvers and particle pushers.  The laser profile rises rapidly, is flat over a large region, and then falls rapidly. The rise and fall taper off smoothly to zero.   This is necessary to avoid unphysical results from the rise and fall of the laser.

The two field solvers available are the Yee and Fei solvers.  The Yee solver is a standard among PIC codes, but has an imperfect dispersion relation for light waves in vacuum.  The Fei (yes, he is your TA) solver uses a 16-point stencil in the longitudinal direction to correct for dispersion errors in solving Maxwell's equations as well as for offset electric and magnetic fields in time.

The six particle pushers are Boris (`standard`), Vay (`vay`), conditional Vay (`cond_vay`), Cary (`cary`), full rotation (`fullrot`), Euler (`euler`), and Pétri (`petri`).  The standard Boris push is the most common used in PIC codes.  The Vay pusher preserves the $\vec{E}\times\vec{B}$ velocity but is not volume-preserving, and is useful when particles attain very high energies (for large values of $A_0$); it can also lead to the magnetic field doing work.  The conditional Vay pusher uses the standard Boris push for particles with $\gamma<5$ and the Vay pusher otherwise.  The Cary pusher preserves the $\vec{E}\times\vec{B}$ velocity like the Vay pusher, but it is also volume-preserving, like the standard Boris push.  The full rotation and Euler pushers are like the standard Boris push but perform the $\vec B$ field rotation exactly, and should be identical for our purposes.

For this notebook, we like to experiment to understand the theoretical trajectories, gain confidence in the PIC codes, and to understand some limitations.

Several examples that have proved insightful include the following:

* Yee, $t_f=40$, $u_{z0}=0$, $a_0=0.5$, standard pusher.
    * Do other pushers make much of a difference?
    * Does the Fei solver make much of a difference?
* Yee, $t_f=300$, $u_{z0}=0$, $a_0=5$
    * Try other pushers.  Is any one significantly better?  Significantly worse?
    * Try halving $dt$. Can you get it to converge?  Think about the dispersion relation using the Yee solver as you do this.
    * Does the Fei solver make much of a difference?
* Change $u_{z0}=3$ in the above example and repeat.  **This makes a big difference!**
    * Which pushers give the right answer when using $dt=1$ with the Fei solver?
* Yee/Fei, $t_f=600$, $u_{z0}=20$, $a_0=50$, standard, Vay, Cary, and full rotation pushers
    * The trends in this example are all amplified compared to the other cases.  How important is the Fei solver for this case?  What about the different pushers?
    * Try reducing $dt$ for the Yee solver with Cary pusher.  Can you get as good agreement for any time step value as you can when using the Fei solver and Cary pusher with $dt=1$?  Think about the dispersion relation for the Yee solver with a small time step.

OSIRIS simulations are done in normalized units. 

* Time:  $t' = t \omega_{p}$

* Frequency: $\omega' = \frac{\omega}{\omega_{p}}$ 

* Position: $\vec{x}' = \frac{\omega_{p}}{c} \vec{x}$  

* Momenta: $\vec{u}' = \frac{\vec{p}}{m_{e} c} = \frac{\vec{\gamma v}}{c} = \frac{\vec{u}} {c}$  

* Electric field: $\vec{E}' = e \frac{c / \omega_{p}}{m_{e} c^2} \vec{E}$  

* Magnetic field: $\vec{B}' = e \frac{c / \omega_{p}}{m_{e} c^2} \vec{B}$  


In [ ]:
# **********************************
# First Run This Cell to 
# import the necessary libraries
# **********************************
#
# Please run this cell before running anything, and run this cell again if you have restarted the 
# python kernel.
#
# This cell imports useful routines to diagnose the simulations here
#

import h5py
%matplotlib notebook
import numpy as np
import sys, os
import matplotlib.pyplot as plt
from single_particle_helper import haines

In [ ]:
def run_plot_std( a0_in=None, p10_in=None,
                 names = ['boris','vay','cary','fullrot','euler','petri'],
                 t_max=None, run_sims=True, save_fig=True):
    
    markers = ['v','o','^','x','+']
    titles = [r'$\xi$ $[c/\omega_0]$',r'$x$ $[c/\omega_0]$',r'$p_z$ $[m_ec]$',r'$p_x$ $[m_ec]$',r'$\gamma$']
    
    fig, ax = plt.subplots(2,3,figsize=(10,8))
    ax = ax.flatten()

    for k,name in enumerate(names):
        if run_sims:
            print("Running "+name)
            if a0_in==None:
                if p10_in!=None:
                    print("a0_in and p10_in must either both be given or both ignored")
                os.system('python pushers.py '+name)
            else:
                if p10_in==None:
                    print("a0_in and p10_in must either both be given or both ignored")
                os.system('python pushers.py '+name+' '+str(a0_in)+' '+str(p10_in))

        dat=np.load('single-part-'+name+'.npz')
        dt = dat['dt']
        t = np.linspace(0,dt*dat['n_steps'],dat['n_steps']+1)
        p10 = dat['p10']
        a0 = dat['a0']
        phi0 = dat['phi0']

        x1 = dat['x'][0,:,:]
        x2 = dat['x'][1,:,:]
        p1 = dat['p'][0,:,:]
        p2 = dat['p'][1,:,:]
        gamma = dat['gamma'][:,:]
        all_data = [x1,x2,p1,p2,gamma]
        npart = x1.shape[0]
        err_std = np.zeros((npart,len(all_data)))

        x0s = x1[:,0]

        for ll in np.arange(dat['x'].shape[1]):

            uz0 = p1[ll,0];

            if t_max==None or t_max >= np.max(t):
                tf = np.max(t)
                l = len(t) - 1
            else:
                tf = t_max
                l = np.argmax(t>t_max)

            ux0=0.0; uy0=0.0; t0=phi0-x0s[ll]; z0=0.0;
            [tt,xx,zz,pxx,pzz,gg] = haines(a0,ux0,uy0,uz0,t0,tf,z0)
            all_analytical = [zz+x0s[ll],xx,pzz,pxx,gg]
            for i in range(len(all_data)):
                err_std[ll,i] = np.sqrt( np.mean( np.square( 
                                all_data[i][ll,:l] - np.interp(t[:l],tt,all_analytical[i]) ) ) )
                # err_end[ll,k] = all_analytical[i][-1] - all_data[i][ll,l]

        for i in range(len(all_data)):
            ax[i].plot(x0s,err_std[:,i],'-'+markers[k])
            if k==0:
                ax[i].set_xlabel('Initial position')
                ax[i].set_ylabel('Error in '+titles[i])

    ax[0].legend(names)
    ax[-1].axis('off')
    plt.suptitle('$a_0='+str(a0)+'$, $p_{z,0}='+str(p10)+'$, $\Delta t='+str(dt)+'$, $T_f={:.1f}$'.format(tf))
    plt.tight_layout(rect=[0,0,1,0.97])
    if save_fig:
        plt.savefig('error-a0-{}-pz0-{}.png'.format(a0,p10),dpi=300)
    plt.show()

In [ ]:
def run_plot_traj( a0_in=None, p10_in=None, n_plot_in=5,
                 names = ['boris','vay','cary','fullrot','euler','petri'],
                 t_max=None, run_sims=True, save_fig=True, plot_z=False ):
    
    if plot_z:
        titles = [r'$z$ $[c/\omega_0]$',r'$x$ $[c/\omega_0]$',r'$p_z$ $[m_ec]$',r'$p_x$ $[m_ec]$',r'$\gamma$']
    else:
        titles = [r'$\xi$ $[c/\omega_0]$',r'$x$ $[c/\omega_0]$',r'$p_z$ $[m_ec]$',r'$p_x$ $[m_ec]$',r'$\gamma$']
    markers = ['v','o','^','x','+']

    for k,name in enumerate(names):
        if run_sims:
            print("Running "+name)
            if a0_in==None:
                if p10_in!=None:
                    print("a0_in and p10_in must either both be given or both ignored")
                os.system('python pushers.py '+name)
            else:
                if p10_in==None:
                    print("a0_in and p10_in must either both be given or both ignored")
                os.system('python pushers.py '+name+' '+str(a0_in)+' '+str(p10_in))

        dat=np.load('single-part-'+name+'.npz')
        dt = dat['dt']
        t = np.linspace(0,dt*dat['n_steps'],dat['n_steps']+1)
        p10 = dat['p10']
        a0 = dat['a0']
        phi0 = dat['phi0']

        x1 = dat['x'][0,:,:]
        x2 = dat['x'][1,:,:]
        p1 = dat['p'][0,:,:]
        p2 = dat['p'][1,:,:]
        gamma = dat['gamma'][:,:]
        if plot_z:
            all_data = [x1,x2,p1,p2,gamma]
        else:
            all_data = [t-x1,x2,p1,p2,gamma]
        
        if k==0:
            npart = x1.shape[0]
            if n_plot_in > npart:
                n_plot = npart
            else:
                n_plot = n_plot_in
            
            inds = np.linspace(0,npart-1,n_plot).round().astype(int)
            fig, ax = plt.subplots( n_plot, 5, figsize=(10,4*n_plot), squeeze=False)
            
            if t_max==None or t_max >= np.max(t):
                tf = np.max(t)
                l = len(t) - 1
            else:
                tf = t_max
                l = np.argmax(t>t_max)

        for i,ind in enumerate(inds):
            
            for j in range(len(all_data)):
                ax[i,j].plot(t[:l],all_data[j][ind,:l],label=names[k])

            if k==len(names)-1:
                uz0 = p1[ind,0];

                ux0=0.0; uy0=0.0; t0=phi0-x1[ind,0]; z0=0.0;
                [tt,xx,zz,pxx,pzz,gg] = haines(a0,ux0,uy0,uz0,t0,tf,z0)
                if plot_z:
                    all_analytical = [zz+x1[ind,0],xx,pzz,pxx,gg]
                else:
                    all_analytical = [tt-zz-x1[ind,0],xx,pzz,pxx,gg]

                for j in range(len(all_data)):
                    ax[i,j].plot(tt,all_analytical[j],'--',label='theory')
                    ax[i,j].set_ylabel(titles[j])
                    ax[i,j].set_xlabel('$t$ $[\omega_0^{-1}]$')

                if plot_z:
                    prefix=r'$z'
                else:
                    prefix=r'$\xi'
                ax[i,0].text( 0.35, 0.05, prefix+r'_0={:.2f}$'.format(all_data[0][ind,0]),
                             transform=ax[i,0].transAxes, bbox=dict(facecolor='none', edgecolor='k'),
                            fontsize=8 )

    ax[0,0].legend(fontsize=8)
    plt.suptitle('$a_0='+str(a0)+'$, $p_{z,0}='+str(p10)+'$, $\Delta t='+str(dt)+'$, $T_f={:.1f}$'.format(tf))
    plt.tight_layout(rect=[0,0,1,0.97])
    if save_fig:
        plt.savefig('trajectories-a0-{}-pz0-{}-npart-{}.png'.format(a0,p10,n_plot),dpi=300)
    plt.show()

In [ ]:
pzs = [0,0.01,0.03,0.1,0.3,1,3,10,30]
a0s = [1,3,6,9]

for p10 in pzs:
    for a0 in a0s:

        run_plot_std( a0_in=a0, p10_in=p10, run_sims=True, save_fig=True )

In [ ]:
pzs = [0,0.01,0.03,0.1,0.3,1,3,10,30]
a0s = [1,3,6,9]

for p10 in pzs:
    for a0 in a0s:

        run_plot_traj( a0_in=a0, p10_in=p10, n_plot_in=5 )